<a href="https://colab.research.google.com/github/GowthamSid727/Carvana-detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch 
import torch.nn as nn
import numpy as np
from torchsummary import summary

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [3]:
class encoder_conv(nn.Module):
    def __init__(self,input,output):
        super(encoder_conv,self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(input,output,kernel_size=3,stride=1),
            nn.BatchNorm2d(output),
            nn.ReLU(),
            nn.Conv2d(output,output,kernel_size=3,stride=1),
            nn.BatchNorm2d(output),
            nn.ReLU(),
        )
    
    def forward(self,x):
        x = self.double_conv(x)
        return x

In [10]:
class Unet(nn.Module):
    def __init__(self,input,output):
        super(Unet,self).__init__()
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.downsample_1 = encoder_conv(input,64) 
        self.downsample_2 = encoder_conv(64,128) 
        self.downsample_3 = encoder_conv(128,256)
        self.downsample_4 = encoder_conv(256,512)
        self.bottelneck = encoder_conv(512,1024) 
        self.upscale_1 = nn.ConvTranspose2d(1024,512,kernel_size=2,stride=2)
        self.upconv_1 = encoder_conv(1024,512) 
        self.upscale_2 = nn.ConvTranspose2d(512,256,2,2)
        self.upconv_2 = encoder_conv(512,256)
        self.upscale_3 = nn.ConvTranspose2d(256,128,2,2)
        self.upconv_3 = encoder_conv(256,128)
        self.upscale_4 = nn.ConvTranspose2d(128,64,2,2)
        self.upconv_4 = encoder_conv(128,64)
        self.fc_conv = nn.Sequential(nn.Conv2d(64,1,2,2),
                                     nn.Sigmoid()
                                     )
    def forward(self,x):
        x1 = self.downsample_1(x) # (3,572,572) -> (64,568,568)
        x2 = self.maxpool(x1) # (64,568,568)->(64,284,284)
        x3 = self.downsample_2(x2)# (64,284,284) -> (128,280,280)
        x4 = self.maxpool(x3) #(128,280,280) -> (128,140,140)
        x5 = self.downsample_3(x4) # (128,140,140) -> (256,136,136)
        x6 = self.maxpool(x5) # (256,136,136) -> (256,68,68)
        x7 = self.downsample_4(x6) # (256,68,68) -> (512,64,64)
        x8 = self.maxpool(x7) #(512,64,64) -> (512,32,32)
        x9 = self.bottelneck(x8) # (512,32,32) -> (1024,28,28)
        #upscale
        x = self.upscale_1(x9) #(1024,28,28) ->  (512, 56, 56)
        y  = x7[:,:,:x.shape[2],:x.shape[2]]
        x11 = self.upconv_1(torch.cat((y,x),1)) # torch.cat -> (1024, 56, 56) -> (512, 52, 52)
        x = self.upscale_2(x11) # 512, 52, 52 -> 256, 104, 104
        y = x5[:,:,:x.shape[2],:x.shape[2]]
        x12 = self.upconv_2(torch.cat((y,x),1)) # torch.cat -> 512, 104, 104 ->  256, 100, 100
        x = self.upscale_3(x12)
        y = x3[:,:,:x.shape[2],:x.shape[2]]
        x13 = self.upconv_3(torch.cat((y,x),1))
        x = self.upscale_4(x13) 
        y = x1[:,:,:x.shape[2],:x.shape[2]]
        x14 = self.upconv_4(torch.cat((y,x),1))
        x15 = self.fc_conv(x14)
        return x15

In [11]:
model = Unet(1,2).to(device=device)
image = np.random.randn(1,1,480,480)
image = torch.FloatTensor(image)
image = image.to(device)

In [12]:
model(image).shape

torch.Size([1, 1, 146, 146])

In [17]:
print(model)

Unet(
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (downsample_1): encoder_conv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (downsample_2): encoder_conv(
    (double_conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (downsample_3): encoder_conv(
    (double_conv): Sequential(
      (0): Co